In [6]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from gensim.models import word2vec

/Users/mmiyazaki/dataiku/Design/DATA_DIR/code-envs/python/japan-nlp/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [22]:
# Read recipe inputs
model_folder_path = dataiku.Folder("m9JZdV7b").get_path()
model_path = model_folder_path + "/word2vec_ramen_model.model"
ramen_model = word2vec.Word2Vec.load(model_path)

In [39]:
reviews_TF_IDF = dataiku.Dataset("reviews_TF_IDF")
df = reviews_TF_IDF.get_dataframe()

In [36]:
tripAdvisor_path = dataiku.Folder("FXMfF0DU").get_path() + "/TripAdvisor_top_TFIDF_words.txt"
f = open(tripAdvisor_path,'r',encoding="utf-8")

In [37]:
words_ta = []
for i,data in enumerate(f):
    word = data.replace("'",'').replace('[','').replace(']','').replace(' ','').replace('\n','')
    words_ta.append(word)

In [38]:
words_ta

['レストラン',
 '私たち',
 'あなた',
 '彼ら',
 'パリ',
 '待ち行列',
 'クール',
 '快適',
 '窮屈',
 '事項',
 'キリン',
 '共有',
 'パイ',
 '宣伝',
 '提案',
 'バー',
 'インテリア',
 'コンパクト',
 'ストック',
 '親切']

In [40]:
df

,store_name,address,ward,score,review_cnt,review,texts_tfidf_sorted_top20,id
0,手打式超多加水麺 ののくら,東京都葛飾区亀有3-11-11マーベラス大協ビル 1F,東京都内,3.98,479,2020年、ラーメン103杯目(^^)本日、朝イチで営業を済ませ、次の打ち合わせまで時間が空...,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時...",ID-000001
1,Homemade Ramen 麦苗,東京都品川区南大井6-11-10,東京都内,3.97,731,2020/08/03(月)5度目の訪問！現在は蜜を避けるために、事前記帳(予約)式になってい...,"['記帳', 'ボード', 'ウェイティング', '深谷', '9時', '記名', 'ワン...",ID-000002
2,麺尊 RAGE,東京都杉並区松庵3-37-22レンツェン松庵 1F,東京都内,3.96,723,5年連続で「ミシュランガイド東京」のビブグルマンに掲載され、「The Tabelog Awa...,"['金華', 'ハム', 'ロック', '穂先', '燻製', 'レモン', '豚バラ', ...",ID-000003
3,宍道湖しじみ中華蕎麦 琥珀,東京都大田区西六郷2-1-3,東京都内,3.95,296,本日は貴重な平日のお休みだった為、普段は中々足を運ぶ事の出来ない蒲田方面へ。11時40分に蒲...,"['シジミ', 'バラ', '焼き', '紫玉ねぎ', '吊し', '菅野', 'ワンタン'...",ID-000004
4,らぁ麺や 嶋,東京都渋谷区本町3-41-12,東京都内,3.95,177,今年、西新宿五丁目に新しいラーメン屋が出来た。まだ1年も経っていないだろうにすでに期待度が非...,"['記帳', '4点', '細切り', '記入', '炭火焼き', '昆布', '燻製', ...",ID-000005
...,...,...,...,...,...,...,...,...
206,麺屋はやしまる,東京都杉並区高円寺北2-22-111F,東京都内,3.71,297,東京都高円寺駅北口から徒歩5分の麺屋 はやしまるにて、わんたん2コと煮玉子入りつけめん塩99...,"['ワンタン', 'なめこ', 'きゅうり', '炸醤', '先払い', '搾菜', '冷や...",ID-000207
207,らーめん紬麦,東京都千代田区神田平河町1第三東ビル B1F,東京都内,3.71,327,初訪問です。前から行きたかったのですが、なかなか行く機会がなく、今日は近くに行く予定があった...,"['蒸し鶏', '階段', 'おば', 'ナルト', 'ピリ辛', 'ゲンコツ', '和風'...",ID-000208
208,づゅる麺池田,東京都目黒区目黒1-6-12,東京都内,3.71,452,http://wineninja.net/tabearuki-0443/ようやく落ち着いてき...,"['レモン', 'キャベツ', 'つけ出', '国産', '秋刀魚', '宣言', 'サラ'...",ID-000209
209,つじ田 味噌の章,東京都千代田区神田小川町1-1イルヴィアーレ神田小川町ビル 1F,東京都内,3.71,418,神田小川町で午前中の打ち合わせを終え、小川町の交差点付近にあるつじ田さんへ。つけ麺で有名なつ...,"['青のり', '白味噌', '青海苔', '七味', 'ドロ', 'ちぢれ麺', '挽肉'...",ID-000210


In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

recommendation_df = reviews_TF_IDF_df # For this sample code, simply copy input to output


# Write recipe outputs
recommendation = dataiku.Dataset("recommendation")
recommendation.write_with_schema(recommendation_df)